In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
data1 = pd.read_csv('result/rsrp-combined-alljabo-40x40.csv/0.part')
polygon = gpd.read_file('Polygon XL - EID Area/Kecamatan_Jabotabek.TAB')

In [3]:
data1   = gpd.GeoDataFrame(data1, geometry=gpd.GeoSeries.from_wkt(data1['geometry_polygon']), crs=4326)

In [4]:
data1 = gpd.sjoin(left_df= data1, right_df=polygon, how='inner', predicate='within')

In [5]:
data1.drop(columns=['geometry', 'index_right', 'ID_Kecamatan', 'Luas_Kec_Sq_Km', 'Unique_XL', 'Kecamatan_XL', 'City_XL', 'Commercial_XL', 'Clutter_Type'], inplace=True)

In [6]:
data1.to_csv('result/result-combined-rsrp.csv')